In [1]:
# import os.path

# from google.auth.transport.requests import Request
# from google.oauth2.credentials import Credentials
# from google_auth_oauthlib.flow import InstalledAppFlow
# from googleapiclient.discovery import build
# from googleapiclient.errors import HttpError

# # If modifying these scopes, delete the file token.json.
# SCOPES = ["https://www.googleapis.com/auth/youtube.readonly","https://www.googleapis.com/auth/youtube.force-ssl"]


# creds = None
# # The file token.json stores the user's access and refresh tokens, and is
# # created automatically when the authorization flow completes for the first
# # time.
# if os.path.exists("token.json"):
#   creds = Credentials.from_authorized_user_file("token.json", SCOPES)
# # If there are no (valid) credentials available, let the user log in.
# if not creds or not creds.valid:
#   if creds and creds.expired and creds.refresh_token:
#     creds.refresh(Request())
#   else:
#     flow = InstalledAppFlow.from_client_secrets_file(
#         "credentials.json", SCOPES
#     )
#     creds = flow.run_local_server(port=0)
#   # Save the credentials for the next run
#   with open("token.json", "w") as token:
#     token.write(creds.to_json())

# try:
#   youtube = build("youtube", "v3", credentials=creds)

#       # Taking input from the user and slicing for video id
#   video_id = input('Enter Youtube Video URL: ')[-11:]
#   print("video id: " + video_id)
  
#   # Getting the channelId of the video uploader
#   video_response = youtube.videos().list(
#       part='snippet',
#       id=video_id
#   ).execute()
  
#   print(video_response)
#   # Splitting the response for channelID
#   video_snippet = video_response['items'][0]['snippet']
#   uploader_channel_id = video_snippet['channelId']
#   print("channel id: " + uploader_channel_id)
  
# except HttpError as err:
#   print(err)


In [2]:
from googleapiclient.discovery import build
import re

API_KEY = ''  # Put in your API Key

youtube = build('youtube', 'v3', developerKey=API_KEY)  # initializing Youtube API

# Improved input handling for video ID
def extract_video_id(url):
    match = re.search(r'(?:v=|\/)([0-9A-Za-z_-]{11}).*', url)
    return match.group(1) if match else None

video_url = input('Enter YouTube Video URL: ')
video_id = extract_video_id(video_url)

if not video_id:
    print("Invalid YouTube URL")
else:
    print("video id: " + video_id)

    # Getting the channelId of the video uploader
    try:
        video_response = youtube.videos().list(
            part='snippet',
            id=video_id
        ).execute()

        print("Video Response:", video_response)  # Debugging step

        if 'items' in video_response and video_response['items']:
            video_snippet = video_response['items'][0]['snippet']
            uploader_channel_id = video_snippet.get('channelId', 'No channelId found')
            print("channel id: " + uploader_channel_id)

            # Fetch comments
            print("Fetching Comments...")
            comments = []
            nextPageToken = None
            while len(comments) < 600:
                request = youtube.commentThreads().list(
                    part='snippet',
                    videoId=video_id,
                    maxResults=100,  # You can fetch up to 100 comments per request
                    pageToken=nextPageToken
                )
                response = request.execute()
                for item in response['items']:
                    comment = item['snippet']['topLevelComment']['snippet']
                    # Check if the comment is not from the video uploader
                    if comment['authorChannelId']['value'] != uploader_channel_id:
                        comments.append(comment['textDisplay'])
                nextPageToken = response.get('nextPageToken')

                if not nextPageToken:
                    break
            # Print the first 5 comments
            print(comments[:5])

        else:
            print("No video found for the provided ID.")
    except Exception as e:
        print("An error occurred:", e)


video id: Pv0iVoSZzN8
Video Response: {'kind': 'youtube#videoListResponse', 'etag': 'wBOw43Neh-kQLqMfCKOqLW-OJFE', 'items': [{'kind': 'youtube#video', 'etag': 'pufOP10W29zOstMUtY5vf8fP9cY', 'id': 'Pv0iVoSZzN8', 'snippet': {'publishedAt': '2024-04-27T16:00:00Z', 'channelId': 'UCX6OQ3DkcsbYNE6H8uQQuVA', 'title': 'In 10 Minutes This Room Will Explode!', 'description': 'I didn’t expect that to happen…\nThere’s no jumping through hoops (or plate glass windows) with T-Mobile. Customers can get Magenta Status from day one, including hotel discounts on select brands, deals on rental cars, discounts on select concert tickets nationwide, and so much more. See how at https://t-mobile.com/status\n\nNew Merch - https://mrbeast.store\n\nCheck out Viewstats! - https://www.viewstats.com/\n\nSUBSCRIBE OR I TAKE YOUR DOG\n╔═╦╗╔╦╗╔═╦═╦╦╦╦╗╔═╗\n║╚╣║║║╚╣╚╣╔╣╔╣║╚╣═╣ \n╠╗║╚╝║║╠╗║╚╣║║║║║═╣\n╚═╩══╩═╩═╩═╩╝╚╩═╩═╝\n\nFor any questions or inquiries regarding this video, please reach out to chucky@mrbeastbusiness.c

# Channel to CsV

In [3]:
import csv
from googleapiclient.discovery import build


# Get the channel name and upload playlist ID
def get_channel_info(channel_id):
    response = youtube.channels().list(
        part='snippet,contentDetails',
        id=channel_id
    ).execute()
    if 'items' in response and len(response['items']) > 0:
        channel = response['items'][0]['snippet']['title']
        upload_playlist_id = response['items'][0]['contentDetails']['relatedPlaylists']['uploads']
        return channel, upload_playlist_id
    return None, None

# Fetch 100 video IDs from the uploads playlist
def get_video_ids_from_channel(upload_playlist_id, max_results=100):
    video_ids = []
    next_page_token = None

    while len(video_ids) < max_results:
        response = youtube.playlistItems().list(
            part='snippet',
            playlistId=upload_playlist_id,
            maxResults=50,  # API allows max 50 results per request
            pageToken=next_page_token
        ).execute()
        
        for item in response['items']:
            video_ids.append(item['snippet']['resourceId']['videoId'])

        next_page_token = response.get('nextPageToken')
        if not next_page_token or len(video_ids) >= max_results:
            break
    
    return video_ids[:max_results]

# Fetch the video title
def get_video_title(video_id):
    response = youtube.videos().list(
        part='snippet',
        id=video_id
    ).execute()
    if 'items' in response and len(response['items']) > 0:
        return response['items'][0]['snippet']['title']
    return None

# Fetch comments for a video
def get_comments(video_id, max_comments=100):
    comments = []
    next_page_token = None

    while len(comments) < max_comments:
        response = youtube.commentThreads().list(
            part='snippet',
            videoId=video_id,
            maxResults=100,  # Fetch up to 100 comments per request
            pageToken=next_page_token
        ).execute()

        for item in response['items']:
            comment = item['snippet']['topLevelComment']['snippet']['textDisplay']
            comments.append(comment)

        next_page_token = response.get('nextPageToken')
        if not next_page_token or len(comments) >= max_comments:
            break

    return comments[:max_comments]

# Write the data to a CSV file
def write_to_csv(channel_name, video_data):
    with open(f'{channel_name}_video_comments.csv', mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(["Channel Name", "Video Title", "Comment"])
        
        for video_title, comments in video_data.items():
            for comment in comments:
                writer.writerow([channel_name, video_title, comment])

# Main Function
def fetch_videos_and_comments(channel_id, max_videos=100, max_comments_per_video=100):
    # Get channel information
    channel_name, upload_playlist_id = get_channel_info(channel_id)
    
    if not channel_name or not upload_playlist_id:
        print("Invalid channel ID or channel information not found.")
        return

    print(f"Fetching data for channel: {channel_name}")

    # Get video IDs
    video_ids = get_video_ids_from_channel(upload_playlist_id, max_results=max_videos)

    video_data = {}

    # For each video, get the title and comments
    for video_id in video_ids:
        video_title = get_video_title(video_id)
        if video_title:
            print(f"Fetching comments for video: {video_title}")
            comments = get_comments(video_id, max_comments=max_comments_per_video)
            video_data[video_title] = comments

    # Write all data to a CSV
    write_to_csv(channel_name, video_data)
    print(f"Data saved to {channel_name}_video_comments.csv")




# Channel to Csv threading

In [4]:
import time
start_time = time.time()
# Replace with the actual YouTube channel ID
channel_id = f'UCX6OQ3DkcsbYNE6H8uQQuVA'
fetch_videos_and_comments(channel_id, max_videos=100, max_comments_per_video=100)
end_time = time.time()
execution_time = start_time - end_time
print("Execution time:",execution_time)
start_time = time.time()
fetch_videos_and_comments_opti(channel_id, max_videos=100, max_comments_per_video=100)
end_time = time.time()
execution_time = start_time - end_time
print("Execution time:",execution_time)

Fetching data for channel: MrBeast
Fetching comments for video: Running With Bigger And Bigger Lunchlys
Fetching comments for video: Holding Bigger And Bigger Dogs
Fetching comments for video: Men Vs Women Survive The Wilderness For $500,000
Fetching comments for video: Will A Guitar Boat Hold My Weight?
Fetching comments for video: Real Or Cake For $10,000
Fetching comments for video: 7 Days Stranded In A Cave
Fetching comments for video: Running With Bigger And Bigger Feastables
Fetching comments for video: Survive 100 Days In Nuclear Bunker, Win $500,000
Fetching comments for video: Spot The Fake Animal For $10,000
Fetching comments for video: 50 YouTubers Fight For $1,000,000
Fetching comments for video: Pass The Phone To…
Fetching comments for video: How Many Balloons Does It Take To Fly?
Fetching comments for video: I Built 100 Houses And Gave Them Away!
Fetching comments for video: World’s Deadliest Obstacle Course!
Fetching comments for video: $10,000 Every Day You Survive In T

NameError: name 'fetch_videos_and_comments_opti' is not defined